<a href="https://colab.research.google.com/github/dashang/LangChainAgentWithFactChecker/blob/main/%5B14th_Dec%5D_GenAi_with_Tool_Fact_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y langchain langchain-core langchain-community langchain-google-genai
!pip install langchain==0.1.0 langchain-community==0.0.10 langchain-core==0.1.8 langchain-google-genai==0.0.6 google-search-results
!pip install -q wikipedia

Found existing installation: langchain 1.1.3
Uninstalling langchain-1.1.3:
  Successfully uninstalled langchain-1.1.3
Found existing installation: langchain-core 1.1.3
Uninstalling langchain-core-1.1.3:
  Successfully uninstalled langchain-core-1.1.3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 19.8 MB/s eta 0:00:00

  Preparing metadata (setup.py) ... done


In [5]:

import os
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType
from langchain_community.utilities import SerpAPIWrapper
from google.colab import userdata
import wikipedia

# --- CONFIGURATION ---
# 1. Google Gemini Key
os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_API_KEY')

# 2. SerpAPI Key (Get free at serpapi.com)
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

# --- CHECK KEYS ---
if not os.environ.get("GOOGLE_API_KEY") or os.environ["GOOGLE_API_KEY"] == "YOUR_GOOGLE_GEMINI_KEY":
    print("⚠️ STOP: Please enter your Google API Key!")
elif not os.environ.get("SERPAPI_API_KEY") or os.environ["SERPAPI_API_KEY"] == "YOUR_SERPAPI_KEY":
    print("⚠️ STOP: Please enter your SerpAPI Key!")
else:
    try:
        # --- DEFINE TOOLS ---
        # 1. WIKI Web Search
        # search = SerpAPIWrapper()

        # @tool
        # def search_web(query: str):
        #     """Useful for searching the internet for current events, prices, and weather."""
        #     print(f"    🔎 [Search Tool] Googling: '{query}'")
        #     return search.run(query)

        @tool
        def search_wiki(query: str):
            """Useful for searching the Wikipedia for information."""
            print(f"    🔎 [Wikipedia Tool] Query: '{query}'")
            return wikipedia.summary(query,sentences = 2)



        # 2. Calculator
        @tool
        def calculator(expression: str):
            """Useful for calculating math expressions."""
            print(f"    🧮 [Math Tool] Calculating: '{expression}'")
            try:
                return str(eval(expression))
            except:
                return "Error"

        tools = [search_wiki, calculator]

        # --- INITIALIZE AGENT ---
        llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash") # Using generic pro model for compatibility

        # The "Zero Shot ReAct" agent is the standard "Think-Act-Observe" loop
        agent_executor = initialize_agent(
            tools=tools,
            llm=llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True # Helps if the LLM makes a formatting mistake
        )

        # --- RUN TASK ---
        userQuery = input("Enter your Query: ")
        print("\n--- 🤖 REAL AGENT TASK STARTED ---")

        query = f"You are Expert in Current Affairs and Encyclopedia knowledge, who answer user queries and search on wikipedia for user query: {userQuery} and provide answer from wikipedia. If no answer found mention 'No Answer found'. Show no abussive content"

        # In older versions, we use .run() instead of .invoke()
        response = agent_executor.run(query)

        print(f"\n--- 🏁 FINAL ANSWER ---\n{response}")

    except Exception as e:
        print(f"\n❌ Error: {e}")


Enter your Query: who is putin in russia

--- 🤖 REAL AGENT TASK STARTED ---


> Entering new AgentExecutor chain...
Action: search_wiki
Action Input: Vladimir Putin Russia    🔎 [Wikipedia Tool] Query: 'Vladimir Putin Russia'

Observation: Since 1999, Vladimir Putin has continuously served as either president (acting president from 1999 to 2000; two terms 2000–2008, three terms 2012–present) or Prime Minister of Russia (three months in 1999, full term 2008–2012). Putin has been described as the de facto leader of Russia since 2000.
Thought:I now know the final answer
Final Answer: Vladimir Putin has continuously served as either president or Prime Minister of Russia since 1999. He was acting president from 1999 to 2000, served two terms as president from 2000–2008, and has been president for three terms from 2012–present. He also served as Prime Minister for three months in 1999 and a full term from 2008–2012. He is described as the de facto leader of Russia since 2000.

> Finished chai

In [12]:

import os
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType
from langchain_community.utilities import SerpAPIWrapper
from google.colab import userdata
import wikipedia

# --- CONFIGURATION ---
# 1. Google Gemini Key
os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_API_KEY')


# --- CHECK KEYS ---
if not os.environ.get("GOOGLE_API_KEY") or os.environ["GOOGLE_API_KEY"] == "YOUR_GOOGLE_GEMINI_KEY":
    print("⚠️ STOP: Please enter your Google API Key!")
elif not os.environ.get("SERPAPI_API_KEY") or os.environ["SERPAPI_API_KEY"] == "YOUR_SERPAPI_KEY":
    print("⚠️ STOP: Please enter your SerpAPI Key!")
else:
    try:
        # --- DEFINE TOOLS ---
        # 1. WIKI Web Search
        # search = SerpAPIWrapper()

        @tool
        def search_wiki(query: str):
            """Useful for searching the Wikipedia for information."""
            print(f"    🔎 [Wikipedia Tool] Query: '{query}'")
            return wikipedia.summary(query,sentences = 2)

        # 2. Calculator
        @tool
        def calculator(expression: str):
            """Useful for calculating math expressions."""
            print(f"    🧮 [Math Tool] Calculating: '{expression}'")
            try:
                return str(eval(expression))
            except:
                return "Error"

        tools = [search_wiki, calculator]

        # --- INITIALIZE AGENT ---
        llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash") # Using generic pro model for compatibility

        # The "Zero Shot ReAct" agent is the standard "Think-Act-Observe" loop
        agent_executor = initialize_agent(
            tools=tools,
            llm=llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True # Helps if the LLM makes a formatting mistake
        )

        # --- RUN TASK ---
        userQuery = input("Enter your Query: ")
        print("\n--- 🤖 REAL AGENT TASK STARTED ---")

        query = f"You are Fact checker for science magazine and Wikipedia Expert in knowledge. A writer has submitted draft in user query: {userQuery} and provide answer from wikipedia. If no answer found mention 'No Answer found'. Show no abussive content"
        query =f"""
        You are a fact-checker for a science magazine. UserQuery: {userQuery} . Example : A writer has submitted a draft claiming that 'Isaac Newton lived to be 100 years old.'
        Your Goal:
        Use Wikipedia to find information on UserQuery.
        Use the Calculator to determine the answer if required.
        Declare whether the writer's claim is True or False."""

        # In older versions, we use .run() instead of .invoke()
        response = agent_executor.run(query)

        print(f"\n--- 🏁 FINAL ANSWER ---\n{response}")

    except Exception as e:
        print(f"\n❌ Error: {e}")


Enter your Query: is shahrukh khan female

--- 🤖 REAL AGENT TASK STARTED ---


> Entering new AgentExecutor chain...



❌ Error: Timeout of 60.0s exceeded, last exception: 503 Illegal metadata
